In [1]:
!pip install langchain langgraph langchain-openai faiss-cpu

In [2]:
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# Get the API key from environment variables
openai_api_key = userdata.get("OPENAI_API_KEY")
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [4]:
!pip install langchain_community

In [6]:
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

#from langchain.schema import Document

docs = [
    Document(page_content="High engine temperature may indicate coolant leak or radiator failure."),
    Document(page_content="Low battery voltage often suggests battery degradation."),
    Document(page_content="Brake pressure loss is usually caused by hydraulic fluid leakage."),
    Document(page_content="Brake system faults are safety critical and require immobilization.")
]

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [7]:
from typing import TypedDict, List, Dict

class VehicleState(TypedDict):
    vehicle_id: str
    telemetry: Dict[str, float]
    anomaly: str
    retrieved_docs: List[str]
    diagnosis: str
    decision: str


In [8]:
def telemetry_node(state: VehicleState):
    return state


In [9]:
def anomaly_node(state: VehicleState):
    t = state["telemetry"]

    if t["engine_temp"] > 110:
        state["anomaly"] = "High engine temperature"
    elif t["battery_voltage"] < 11.5:
        state["anomaly"] = "Low battery voltage"
    elif t["brake_pressure"] < 20:
        state["anomaly"] = "Brake pressure loss"
    else:
        state["anomaly"] = "Normal"

    return state


In [10]:
def retrieval_node(state: VehicleState):
    docs = retriever.invoke(state["anomaly"])
    state["retrieved_docs"] = [d.page_content for d in docs]
    return state


In [11]:
def diagnosis_node(state: VehicleState):
    context = "\n".join(state["retrieved_docs"])

    prompt = f"""
    You are an automotive diagnostics expert.

    Telemetry:
    {state["telemetry"]}

    Detected anomaly:
    {state["anomaly"]}

    Relevant knowledge:
    {context}

    Identify the most likely root cause.
    """

    response = llm.invoke(prompt)
    state["diagnosis"] = response.content
    return state


In [12]:
def decision_node(state: VehicleState):
    prompt = f"""
    You are a fleet safety AI.

    Diagnosis:
    {state["diagnosis"]}

    Decide the safest action.
    Constraints:
    - Human safety first
    - Vehicle protection second
    - Cost last

    Respond with ONE action.
    """

    response = llm.invoke(prompt)
    state["decision"] = response.content
    return state


In [13]:
def report_node(state: VehicleState):
    print("\n===== VEHICLE INCIDENT REPORT =====")
    print("Vehicle ID:", state["vehicle_id"])
    print("Telemetry:", state["telemetry"])
    print("Anomaly:", state["anomaly"])
    print("Diagnosis:", state["diagnosis"])
    print("Decision:", state["decision"])
    print("=================================\n")

    return state


In [14]:
from langgraph.graph import StateGraph, END

graph = StateGraph(VehicleState)

graph.add_node("telemetry", telemetry_node)
graph.add_node("anomaly", anomaly_node)
graph.add_node("retrieve", retrieval_node)
graph.add_node("diagnose", diagnosis_node)
graph.add_node("decide", decision_node)
graph.add_node("report", report_node)

graph.set_entry_point("telemetry")

graph.add_edge("telemetry", "anomaly")
graph.add_edge("anomaly", "retrieve")
graph.add_edge("retrieve", "diagnose")
graph.add_edge("diagnose", "decide")
graph.add_edge("decide", "report")
graph.add_edge("report", END)

app = graph.compile()


In [15]:
initial_state = {
    "vehicle_id": "CAR-9001",
    "telemetry": {
        "engine_temp": 128,
        "battery_voltage": 12.1,
        "brake_pressure": 30
    },
    "anomaly": "",
    "retrieved_docs": [],
    "diagnosis": "",
    "decision": ""
}

app.invoke(initial_state)



===== VEHICLE INCIDENT REPORT =====
Vehicle ID: CAR-9001
Telemetry: {'engine_temp': 128, 'battery_voltage': 12.1, 'brake_pressure': 30}
Anomaly: High engine temperature
Diagnosis: Based on the telemetry data and the detected anomaly of high engine temperature, the most likely root cause is related to the cooling system. The engine temperature is significantly high at 128 degrees, which suggests that the engine is not being adequately cooled. This could be due to a coolant leak, which would reduce the coolant level and impair the cooling system's ability to regulate engine temperature. Alternatively, it could be due to a radiator failure, which would also prevent the engine from being properly cooled.

The battery voltage and brake pressure readings do not directly indicate a problem with the cooling system. However, it's important to note that brake system faults are critical, but in this case, the brake pressure reading of 30 does not immediately suggest a fault without additional co

{'vehicle_id': 'CAR-9001',
 'telemetry': {'engine_temp': 128,
  'battery_voltage': 12.1,
  'brake_pressure': 30},
 'anomaly': 'High engine temperature',
 'retrieved_docs': ['High engine temperature may indicate coolant leak or radiator failure.',
  'Brake system faults are safety critical and require immobilization.'],
 'diagnosis': "Based on the telemetry data and the detected anomaly of high engine temperature, the most likely root cause is related to the cooling system. The engine temperature is significantly high at 128 degrees, which suggests that the engine is not being adequately cooled. This could be due to a coolant leak, which would reduce the coolant level and impair the cooling system's ability to regulate engine temperature. Alternatively, it could be due to a radiator failure, which would also prevent the engine from being properly cooled.\n\nThe battery voltage and brake pressure readings do not directly indicate a problem with the cooling system. However, it's important